<h1>Segmenting and Clustering Neighborhoods in Toronto Part I</h1>

<h3> Importing libraries</h3>

In [1]:
import numpy as np # data in a vectorized manner manipulation
import pandas as pd # data analsysis
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # JSON files manipulation
import requests # HTTP library
from bs4 import BeautifulSoup # scraping library

from sklearn.cluster import KMeans # clustering algorithm

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


<h3>Extract data of Toronto neighborhoods from Wikipedia</h3>

In [12]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

text_result = requests.get(url).text #get the entire html of the article as a str
html_parsed_result = BeautifulSoup(text_result, 'html.parser') #transform the text to html
neighbourhood_info_table = html_parsed_result.find('table', class_ = 'wikitable')
neighbourhood_rows = neighbourhood_info_table.find_all('tr')


neighbourhood_info = []
for row in neighbourhood_rows:
    info = row.text.split('\n')[1:-1] 
    neighbourhood_info.append(info)
    
neighbourhood_info[0:10]

[['Postcode', 'Borough', 'Neighbourhood'],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned']]

<h2>Transform the data into a pandas dataframe</h2>

In [13]:
neighbourhood_info[0][-1] = 'Neighbourhood'
neighbourhood_df = pd.DataFrame(neighbourhood_info[1:], columns=neighbourhood_info[0])

neighbourhood_df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


<h2> Dropping the 'Not assigned' Boroughs</h2>

In [14]:
not_assigned_boroughs = neighbourhood_df.index[neighbourhood_df['Borough'] == 'Not assigned']

In [11]:
neighbourhood_df.drop(neighbourhood_df.index[not_assigned_boroughs], inplace=True)
neighbourhood_df.reset_index(drop=True, inplace=True)

neighbourhood_df.head(10)

NameError: name 'neighbourhood_df' is not defined

<h2>Fill the 'Not assigned' neighbourhood with the name of the Borough</h2>

In [ ]:
not_assigned_neighborhoods = neighborhood_df.index[neighborhood_df['Neighbourhood'] == 'Not assigned']

for idx in not_assigned_neighborhoods:
    neighborhood_df['Neighbourhood'][idx] = neighborhood_df['Borough'][idx]
    
neighborhood_df.head(10)

<h2>Combine the grouped Boroughs' neighborhoods separated with a comma</h2>

In [ ]:
group = neighborhood_df.groupby('Postcode')
grouped_neighborhoods = group['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
grouped_boroughs = group['Borough'].apply(lambda x: set(x).pop())
grouped_df = pd.DataFrame(list(zip(grouped_boroughs.index, grouped_boroughs, grouped_neighborhoods)))
grouped_df.columns = ['Postcode', 'Borough', 'Neighbourhood']

grouped_df.head(10)

In [ ]:
print('The DataFrame shape is', grouped_df.shape)